In [79]:
import pandas as pd
import mysql.connector

import numpy as np

### Cargar nuestra base de datos con un csv a partir de la librería mysql.connector 

Antes de cargar los datos, veamos los valores únicos de cada columna para revisar el diseño de la base de datos

In [80]:
# Leer el CSV
df_sale = pd.read_csv("madrid_sale_properties_cleaned.csv")

In [10]:
df_sale.sample(3)

,price_eur,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_consumption_value,energy_emissions_rating,energy_emissions_value,adaptado a personas con movilidad reducida,...,piscina,planta,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles
435,327000.0,Quintana,Ciudad Lineal,40.433745,-3.648607,E,246 kWh/m² año,E,42 Kg CO₂/m² año,0.0,...,False,8.0,False,False,False,51.0,51.0,False,False,False
737,875000.0,Trafalgar,Chamberí,40.432340,-3.703750,False,NaN,False,NaN,0.0,...,False,0.0,False,False,False,98.0,98.0,False,False,False
1605,1900000.0,Castellana,Salamanca,40.438092,-3.688786,False,NaN,False,NaN,0.0,...,False,8.0,False,False,False,170.0,155.0,False,False,False


In [81]:
# Leer el CSV
df_rental = pd.read_csv("madrid_rental_properties_cleaned.csv")

In [12]:
df_rental.sample(3)

,rent_eur_per_month,barrio,distrito,latitude,longitude,listing_type,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,aire acondicionado,...,planta,portero automático,puerta blindada,referencia,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles
1623,2800.0,Palomas,Hortaleza,40.453029,-3.618988,rental,False,False,0.0,False,...,4.0,False,False,I-03003Y-W-03003Y,False,164.0,164.0,False,False,False
2089,1500.0,Valverde,Fuencarral-El Pardo,40.486070,-3.692651,rental,False,False,0.0,True,...,6.0,False,True,22067048,False,110.0,90.0,False,False,False
471,3045.0,Universidad-Malasaña,Centro,40.426859,-3.708706,rental,C,E,0.0,False,...,2.0,False,False,CELEBES,False,140.0,140.0,False,False,False


Identificador de cada piso 

In [17]:
df_rental.shape

(2185, 34)

In [15]:
df_rental["referencia"].unique

<bound method Series.unique of 0                  6013
1                  3893
2                  4411
3                 20815
4       9635001141/9635
             ...       
2180             001913
2181            1101915
2182            1101712
2183            1101954
2184             209599
Name: referencia, Length: 2185, dtype: object>

antigüedad

In [20]:
df_rental["antigüedad"].unique()

array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
       'Entre 5 y 10 años', 'Menos de 5 años', 'Entre 10 y 20 años'],
      dtype=object)

In [21]:
df_sale["antigüedad"].unique()

array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
       'Menos de 5 años', 'Entre 10 y 20 años', 'Entre 5 y 10 años'],
      dtype=object)

conservacion

planta

exterior

In [23]:
df_rental.columns

Index(['rent_eur_per_month', 'barrio', 'distrito', 'latitude', 'longitude',
       'listing_type', 'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles'],
      dtype='object')

In [24]:
rental = ['energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles']

In [25]:
[df_rental[columna].unique() for columna in rental]

[array(['False', 'D', 'E', 'C', 'F', 'G', 'B', 'A'], dtype=object),
 array(['False', 'D', 'E', 'C', 'G', 'F', 'B', 'A'], dtype=object),
 array([0., 1.]),
 array([ True, False]),
 array(['No', 'Si', 'Semi'], dtype=object),
 array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
        'Entre 5 y 10 años', 'Menos de 5 años', 'Entre 10 y 20 años'],
       dtype=object),
 array([False,  True]),
 array(['False', '1.0'], dtype=object),
 array([5, 1, 2, 4, 3, 6, 7, 8], dtype=int64),
 array([ True, False]),
 array(['False', '1.0'], dtype=object),
 array(['True', 'False', 'Independiente', 'Equipada',
        'Amueblada y equipada', 'Equipada e independiente.',
        'Cocina amueblada', 'Cocina americana', 'Completamente equipada',
        'Completamente equipada integrada al salón',
        'Independiente - amueblada con elec.',
        'Independiente totalmente equipada y amueblada', 'Cocina equipada',
        'Americana', 'Americana - amueblada con elec.', 'Amueblada',
 

In [18]:
df_sale.columns

Index(['price_eur', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_consumption_value',
       'energy_emissions_rating', 'energy_emissions_value',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles'],
      dtype='object')

In [33]:
sale = ['energy_consumption_rating',
       'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad',
       'terraza', 'trastero', 'vidrios dobles']

In [34]:
[df_sale[columna].unique() for columna in sale]

[array(['False', 'E', 'C', 'D', 'A', 'F', 'G', 'B'], dtype=object),
 array(['False', 'D', 'E', 'A', 'F', 'C', 'B', 'G'], dtype=object),
 array([0., 1.]),
 array([ True, False]),
 array([ True, False]),
 array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
        'Menos de 5 años', 'Entre 10 y 20 años', 'Entre 5 y 10 años'],
       dtype=object),
 array([False,  True]),
 array(['1.0', 'False'], dtype=object),
 array([ 5,  2,  1,  4,  3,  6,  7, 17,  8, 11,  9, 13, 10, 14],
       dtype=int64),
 array([ True, False]),
 array(['False', 'True', 'Preinstalación'], dtype=object),
 array(['A estrenar', 'En buen estado', 'Reformado', 'A reformar'],
       dtype=object),
 array([False,  True]),
 array([False,  True]),
 array([ 4.,  5.,  2.,  3., 14.,  6.,  1., 12.,  8., nan, 24.,  7., 11.,
         9., 10., 17., 13., 20., 36.]),
 array([False,  True]),
 array([False,  True]),
 array([ 1.,  2.,  3.,  0.,  5.,  6.,  4.,  7.,  8., 10., -1.,  9., 12.,
        11., 20., 14., 16

In [40]:
df_sale['energy_consumption_value'].unique()

array([nan, '151 kWh/m² año', '152 kWh/m² año', '30 kWh/m² año',
       '100 kWh/m² año', '300 kWh/m² año', '219 kWh/m² año',
       '24 kWh/m² año', '124 kWh/m² año', '346 kWh/m² año',
       '95 kWh/m² año', '246 kWh/m² año', '228 kWh/m² año',
       '195 kWh/m² año', '307 kWh/m² año', '130 kWh/m² año',
       '63 kWh/m² año', '309 kWh/m² año', '199 kWh/m² año',
       '148 kWh/m² año', '134 kWh/m² año', '150 kWh/m² año',
       '121 kWh/m² año', '98 kWh/m² año', '169 kWh/m² año',
       '104 kWh/m² año', '179 kWh/m² año', '167 kWh/m² año',
       '298 kWh/m² año', '244 kWh/m² año', '165 kWh/m² año',
       '999 kWh/m² año', '254 kWh/m² año', '242 kWh/m² año',
       '146 kWh/m² año', '226 kWh/m² año', '50 kWh/m² año',
       '361 kWh/m² año', '86 kWh/m² año', '192 kWh/m² año',
       '188 kWh/m² año', '48 kWh/m² año', '196 kWh/m² año',
       '204 kWh/m² año', '75 kWh/m² año', '174 kWh/m² año',
       '198 kWh/m² año', '315 kWh/m² año', '194 kWh/m² año',
       '136 kWh/m² año', '26

In [36]:
df_rental["energy_consumption_rating"].unique()

array(['False', 'D', 'E', 'C', 'F', 'G', 'B', 'A'], dtype=object)

In [41]:
df_rental["latitude"]

0       40.434281
1       40.438612
2       40.435237
3       40.429047
4       40.422607
          ...    
2180    40.516174
2181    40.516126
2182    40.521171
2183    40.519891
2184    40.507050
Name: latitude, Length: 2185, dtype: float64

In [42]:
df_sale.columns

Index(['price_eur', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_consumption_value',
       'energy_emissions_rating', 'energy_emissions_value',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles'],
      dtype='object')

In [43]:
df_rental.columns

Index(['rent_eur_per_month', 'barrio', 'distrito', 'latitude', 'longitude',
       'listing_type', 'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles'],
      dtype='object')

conectamos con la base de datos para hacer la insercion

In [82]:
# Conexión a MySQL
conn = mysql.connector.connect(
    host="localhost", 
    user="root", 
    password="contraseña",
    database="pisos"
)

In [83]:
cursor = conn.cursor()

Una vez conectada la base de datos vamos a concatenar los dos dataframe, para ello necesitamos unificar los campos 

### Unificar precio y tipo

In [84]:
df_rental = df_rental.rename(
    columns={"rent_eur_per_month": "price_current"}
)

In [88]:
df_rental.columns

Index(['price_current', 'barrio', 'distrito', 'latitude', 'longitude',
       'listing_type', 'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles', 'price_kind'],
      dtype='object')

In [85]:
df_rental["price_kind"]   = "rent_month"
df_rental["listing_type"] = "rental"    # para identificarlos mejor 

In [90]:
df_rental["price_kind"].unique()

array(['rent_month'], dtype=object)

In [91]:
df_rental["listing_type"].unique()

array(['rental'], dtype=object)

In [86]:
df_sale = df_sale.rename(
    columns={"price_eur": "price_current"}
)

In [92]:
df_sale.columns

Index(['price_current', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_consumption_value',
       'energy_emissions_rating', 'energy_emissions_value',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles', 'price_kind', 'listing_type'],
      dtype='object')

In [87]:
df_sale["price_kind"]   = "sale_price"
df_sale["listing_type"] = "sale"

In [93]:
df_sale["price_kind"].unique()

array(['sale_price'], dtype=object)

In [94]:
df_sale["listing_type"].unique()

array(['sale'], dtype=object)

### Las columnas de valor energetico de sale no las vamos a usar así que serán borradas

In [95]:
df_sale = df_sale.drop(columns=["energy_consumption_value", "energy_emissions_value"])

In [96]:
df_sale.columns

Index(['price_current', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles', 'price_kind', 'listing_type'],
      dtype='object')

### Necesitamos crear referencias unicas para ventas

In [97]:
df_rental["referencia"]

0                  6013
1                  3893
2                  4411
3                 20815
4       9635001141/9635
             ...       
2180             001913
2181            1101915
2182            1101712
2183            1101954
2184             209599
Name: referencia, Length: 2185, dtype: object

In [99]:
["ref"+str(i) for i in range(len(df_sale))]

['ref0',
 'ref1',
 'ref2',
 'ref3',
 'ref4',
 'ref5',
 'ref6',
 'ref7',
 'ref8',
 'ref9',
 'ref10',
 'ref11',
 'ref12',
 'ref13',
 'ref14',
 'ref15',
 'ref16',
 'ref17',
 'ref18',
 'ref19',
 'ref20',
 'ref21',
 'ref22',
 'ref23',
 'ref24',
 'ref25',
 'ref26',
 'ref27',
 'ref28',
 'ref29',
 'ref30',
 'ref31',
 'ref32',
 'ref33',
 'ref34',
 'ref35',
 'ref36',
 'ref37',
 'ref38',
 'ref39',
 'ref40',
 'ref41',
 'ref42',
 'ref43',
 'ref44',
 'ref45',
 'ref46',
 'ref47',
 'ref48',
 'ref49',
 'ref50',
 'ref51',
 'ref52',
 'ref53',
 'ref54',
 'ref55',
 'ref56',
 'ref57',
 'ref58',
 'ref59',
 'ref60',
 'ref61',
 'ref62',
 'ref63',
 'ref64',
 'ref65',
 'ref66',
 'ref67',
 'ref68',
 'ref69',
 'ref70',
 'ref71',
 'ref72',
 'ref73',
 'ref74',
 'ref75',
 'ref76',
 'ref77',
 'ref78',
 'ref79',
 'ref80',
 'ref81',
 'ref82',
 'ref83',
 'ref84',
 'ref85',
 'ref86',
 'ref87',
 'ref88',
 'ref89',
 'ref90',
 'ref91',
 'ref92',
 'ref93',
 'ref94',
 'ref95',
 'ref96',
 'ref97',
 'ref98',
 'ref99',
 'ref100',

In [100]:
df_sale["referencia"] = ["ref"+str(i) for i in range(len(df_sale))]

In [101]:
df_sale["referencia"]

0          ref0
1          ref1
2          ref2
3          ref3
4          ref4
         ...   
2997    ref2997
2998    ref2998
2999    ref2999
3000    ref3000
3001    ref3001
Name: referencia, Length: 3002, dtype: object

### Definir columnas finales en orden y concatenar

In [104]:
common_cols = [
  "price_current", "price_kind", "listing_type",
  "barrio", "distrito", "latitude", "longitude",
  "energy_consumption_rating", "energy_emissions_rating",
  "adaptado a personas con movilidad reducida",
  "aire acondicionado",
  "amueblado",
  "antigüedad",
  "ascensor", "balcón", "baños", "calefacción", "chimenea",
  "cocina equipada", "conservación", "exterior", "garaje",
  "habitaciones", "jardín", "piscina", "planta",
  "portero automático", "puerta blindada",
  "sistema de seguridad",
  "superficie construida", "superficie útil",
  "terraza", "trastero", "vidrios dobles",
  "referencia"
]

In [105]:
for df in (df_rental, df_sale):
    for col in common_cols:
        if col not in df.columns:
            df[col] = np.nan

In [106]:
df_rental = df_rental[common_cols]
df_sale   = df_sale[common_cols]

In [107]:
df = pd.concat([df_rental, df_sale], ignore_index=True)
df = df.drop_duplicates()

In [108]:
df.shape

(5186, 35)

In [109]:
df_sale.shape + df_rental.shape

(3002, 35, 2185, 35)

In [110]:
df.tail(2)

,price_current,price_kind,listing_type,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,...,planta,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles,referencia
5185,2490000.0,sale_price,sale,Trafalgar,Chamberí,40.435597,-3.700479,False,False,0.0,...,4.0,False,True,False,185.0,160.0,False,False,False,ref3000
5186,2460000.0,sale_price,sale,Sol,Centro,40.415001,-3.706658,False,False,1.0,...,5.0,False,False,False,139.0,139.0,False,False,False,ref3001


### Insertar datos en la base de datos

 La estrategia de trabajo será ir creando los df correspondientes a cada tabla de sql, rellenando las tablas independientes primero. El orden es el siguiente

age_range
– Inserta todos los rangos de antigüedad.
– (Si lo tienes en CSV, léelo primero y haz INSERT sobre age_range.)

feature_catalog
– Inserta tu lista de features (“ascensor”, “balcón”, …).
– También se puede cargar desde CSV si lo tienes en un fichero aparte.

location
– Inserta todos los barrios/distritos.

property
– Ahora que location y age_range existen, puedes insertar las viviendas.
– Usa el location_id y el age_range_id resultantes.

listing
– Con property_id ya presente, carga los anuncios.

rental_term
– Solo haces INSERT si listing_type = 'rental'. Usa el listing_id correspondiente.

energy_certificate
– Solo para las propiedades que tengan certificado, usando su property_id.

property_feature
– Finalmente, para cada tupla (property_id, feature_id, valor) extraída de tu CSV de extras, haces INSERT.

### Tabla ``age_range``

In [112]:
df["antigüedad"].dropna().unique().tolist()

['Más de 50 años',
 'Entre 20 y 30 años',
 'Entre 30 y 50 años',
 'Entre 5 y 10 años',
 'Menos de 5 años',
 'Entre 10 y 20 años']

In [116]:
age_range = df["antigüedad"].dropna().unique().tolist()

In [117]:
age_range

['Más de 50 años',
 'Entre 20 y 30 años',
 'Entre 30 y 50 años',
 'Entre 5 y 10 años',
 'Menos de 5 años',
 'Entre 10 y 20 años']

creamos la sentencia de sql

In [118]:
sql_age = """INSERT INTO age_range(label) VALUES (%s)"""

In [119]:
cursor.executemany(sql_age, [(label,) for label in age_range])

In [120]:
conn.commit()

Guardamos el mapeo para saber que id tiene cada rango

In [125]:
cursor.execute("SELECT * FROM age_range")

In [122]:
cursor.fetchall()

[(6, 'Entre 10 y 20 años'),
 (2, 'Entre 20 y 30 años'),
 (3, 'Entre 30 y 50 años'),
 (4, 'Entre 5 y 10 años'),
 (1, 'Más de 50 años'),
 (5, 'Menos de 5 años')]

In [126]:
age_map = {label: age_range_id for age_range_id, label in cursor.fetchall()}
age_map

{'Entre 10 y 20 años': 6,
 'Entre 20 y 30 años': 2,
 'Entre 30 y 50 años': 3,
 'Entre 5 y 10 años': 4,
 'Más de 50 años': 1,
 'Menos de 5 años': 5}

### Tabla ``feature_map``

INSERT INTO feature_catalog(nombre) VALUES
  ('ascensor'),
  ('balcon'),
  ('calefaccion'),
  ('chimenea'),
  ('exterior'),
  ('garaje'),
  ('piscina'),
  ('trastero'),
  ('jardin'),
  ('adaptado_pmreducida'),
  ('aire_acondicionado'),
  ('puerta_blindada'),
  ('vidrios_dobles'),
  ('sistema_seguridad'),
  ('terraza');

In [127]:
features = [
    'ascensor',
    'balcón',
    'calefacción',
    'chimenea',
    'exterior',
    'garaje',
    'piscina',
    'trastero',
    'jardín',
    'adaptado_pmreducida',
    'aire_acondicionado',
    'puerta_blindada',
    'vidrios_dobles',
    'sistema_seguridad',
    'terraza'
]

In [128]:
sql_feature = """INSERT INTO feature_catalog(nombre) VALUES (%s)"""

In [129]:
cursor.executemany(sql_feature, [(f,) for f in features])

Extraer el mapeo de la base de datos

In [130]:
cursor.execute("SELECT * FROM feature_catalog")

In [131]:
f = cursor.fetchall()
f

[(10, 'adaptado_pmreducida'),
 (11, 'aire_acondicionado'),
 (1, 'ascensor'),
 (2, 'balcón'),
 (3, 'calefacción'),
 (4, 'chimenea'),
 (5, 'exterior'),
 (6, 'garaje'),
 (9, 'jardín'),
 (7, 'piscina'),
 (12, 'puerta_blindada'),
 (14, 'sistema_seguridad'),
 (15, 'terraza'),
 (8, 'trastero'),
 (13, 'vidrios_dobles')]

In [132]:
feature_map = {nombre: feature_id for feature_id, nombre in f}
feature_map

{'adaptado_pmreducida': 10,
 'aire_acondicionado': 11,
 'ascensor': 1,
 'balcón': 2,
 'calefacción': 3,
 'chimenea': 4,
 'exterior': 5,
 'garaje': 6,
 'jardín': 9,
 'piscina': 7,
 'puerta_blindada': 12,
 'sistema_seguridad': 14,
 'terraza': 15,
 'trastero': 8,
 'vidrios_dobles': 13}

### Tabla ``Location``

In [135]:
df[["barrio","distrito"]].drop_duplicates()

,barrio,distrito
0,Castellana,Salamanca
1,Río Rosas,Chamberí
3,Fuente del Berro,Salamanca
4,Justicia-Chueca,Centro
5,Aravaca,Moncloa-Aravaca
...,...,...
2477,Portazgo,Puente de Vallecas
2493,Apóstol Santiago,Hortaleza
2926,Centro Histórico,Vicálvaro
3557,Horcajo,Moratalaz


In [136]:
df_location = df[["barrio","distrito"]].drop_duplicates()
df_location

,barrio,distrito
0,Castellana,Salamanca
1,Río Rosas,Chamberí
3,Fuente del Berro,Salamanca
4,Justicia-Chueca,Centro
5,Aravaca,Moncloa-Aravaca
...,...,...
2477,Portazgo,Puente de Vallecas
2493,Apóstol Santiago,Hortaleza
2926,Centro Histórico,Vicálvaro
3557,Horcajo,Moratalaz


In [141]:
df_location.isna().sum()

barrio      1
distrito    1
dtype: int64

In [142]:
df_location = df_location.dropna()

Pasamos a tupla para poder insertar

In [143]:
locations = [tuple(x) for x in df_location.values]
locations

[('Castellana', 'Salamanca'),
 ('Río Rosas', 'Chamberí'),
 ('Fuente del Berro', 'Salamanca'),
 ('Justicia-Chueca', 'Centro'),
 ('Aravaca', 'Moncloa-Aravaca'),
 ('Rejas', 'San Blas'),
 ('Berruguete', 'Tetuán'),
 ('Simancas', 'San Blas'),
 ('Ibiza', 'Retiro'),
 ('Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'),
 ('Lista', 'Salamanca'),
 ('Recoletos', 'Salamanca'),
 ('Buenavista', 'Carabanchel'),
 ('Puerta del Ángel', 'Latina'),
 ('Pinar del Rey', 'Hortaleza'),
 ('El Viso', 'Chamartín'),
 ('Atocha', 'Arganzuela'),
 ('Almagro', 'Chamberí'),
 ('Peñagrande', 'Fuencarral-El Pardo'),
 ('Niño Jesús', 'Retiro'),
 ('Imperial', 'Arganzuela'),
 ('Trafalgar', 'Chamberí'),
 ('Palacio', 'Centro'),
 ('Valdeacederas', 'Tetuán'),
 ('Estrella', 'Retiro'),
 ('El Cañaveral', 'Vicálvaro'),
 ('Chopera', 'Arganzuela'),
 ('Jerónimos', 'Retiro'),
 ('Arapiles', 'Chamberí'),
 ('Cuatro Caminos', 'Tetuán'),
 ('Almenara', 'Tetuán'),
 ('Opañel', 'Carabanchel'),
 ('San Pascual', 'Ciudad Lineal'),
 ('Universidad-M

In [144]:
sql_loc = """INSERT INTO location (barrio, distrito) VALUES (%s, %s)"""

In [145]:
cursor.executemany(sql_loc, locations)

mapeo de location_id

In [146]:
cursor.execute("SELECT * FROM location")

In [147]:
l = cursor.fetchall()
l

[(1, 'Castellana', 'Salamanca'),
 (2, 'Río Rosas', 'Chamberí'),
 (3, 'Fuente del Berro', 'Salamanca'),
 (4, 'Justicia-Chueca', 'Centro'),
 (5, 'Aravaca', 'Moncloa-Aravaca'),
 (6, 'Rejas', 'San Blas'),
 (7, 'Berruguete', 'Tetuán'),
 (8, 'Simancas', 'San Blas'),
 (9, 'Ibiza', 'Retiro'),
 (10, 'Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'),
 (11, 'Lista', 'Salamanca'),
 (12, 'Recoletos', 'Salamanca'),
 (13, 'Buenavista', 'Carabanchel'),
 (14, 'Puerta del Ángel', 'Latina'),
 (15, 'Pinar del Rey', 'Hortaleza'),
 (16, 'El Viso', 'Chamartín'),
 (17, 'Atocha', 'Arganzuela'),
 (18, 'Almagro', 'Chamberí'),
 (19, 'Peñagrande', 'Fuencarral-El Pardo'),
 (20, 'Niño Jesús', 'Retiro'),
 (21, 'Imperial', 'Arganzuela'),
 (22, 'Trafalgar', 'Chamberí'),
 (23, 'Palacio', 'Centro'),
 (24, 'Valdeacederas', 'Tetuán'),
 (25, 'Estrella', 'Retiro'),
 (26, 'El Cañaveral', 'Vicálvaro'),
 (27, 'Chopera', 'Arganzuela'),
 (28, 'Jerónimos', 'Retiro'),
 (29, 'Arapiles', 'Chamberí'),
 (30, 'Cuatro Caminos', 'Tet

In [148]:
loc_map = {
    (barrio, distrito): location_id
    for location_id, barrio, distrito in l
}

loc_map

{('Castellana', 'Salamanca'): 1,
 ('Río Rosas', 'Chamberí'): 2,
 ('Fuente del Berro', 'Salamanca'): 3,
 ('Justicia-Chueca', 'Centro'): 4,
 ('Aravaca', 'Moncloa-Aravaca'): 5,
 ('Rejas', 'San Blas'): 6,
 ('Berruguete', 'Tetuán'): 7,
 ('Simancas', 'San Blas'): 8,
 ('Ibiza', 'Retiro'): 9,
 ('Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'): 10,
 ('Lista', 'Salamanca'): 11,
 ('Recoletos', 'Salamanca'): 12,
 ('Buenavista', 'Carabanchel'): 13,
 ('Puerta del Ángel', 'Latina'): 14,
 ('Pinar del Rey', 'Hortaleza'): 15,
 ('El Viso', 'Chamartín'): 16,
 ('Atocha', 'Arganzuela'): 17,
 ('Almagro', 'Chamberí'): 18,
 ('Peñagrande', 'Fuencarral-El Pardo'): 19,
 ('Niño Jesús', 'Retiro'): 20,
 ('Imperial', 'Arganzuela'): 21,
 ('Trafalgar', 'Chamberí'): 22,
 ('Palacio', 'Centro'): 23,
 ('Valdeacederas', 'Tetuán'): 24,
 ('Estrella', 'Retiro'): 25,
 ('El Cañaveral', 'Vicálvaro'): 26,
 ('Chopera', 'Arganzuela'): 27,
 ('Jerónimos', 'Retiro'): 28,
 ('Arapiles', 'Chamberí'): 29,
 ('Cuatro Caminos', 'Tetuán'

### Tabla ``property`` 

In [151]:
df[[
    "barrio","distrito",
    "latitude","longitude",
    "referencia",
    "superficie construida","superficie útil",
    "habitaciones","baños","planta",
    "conservación","antigüedad",
    "amueblado","cocina equipada"
]].drop_duplicates()

,barrio,distrito,latitude,longitude,referencia,superficie construida,superficie útil,habitaciones,baños,planta,conservación,antigüedad,amueblado,cocina equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,NaN,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,NaN,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,NaN,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,True,NaN
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,True,NaN
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,True,NaN
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,NaN,True,NaN


In [152]:
df_property = df[[
    "barrio","distrito",
    "latitude","longitude",
    "referencia",
    "superficie construida","superficie útil",
    "habitaciones","baños","planta",
    "conservación","antigüedad",
    "amueblado","cocina equipada"
]].drop_duplicates()

df_property

,barrio,distrito,latitude,longitude,referencia,superficie construida,superficie útil,habitaciones,baños,planta,conservación,antigüedad,amueblado,cocina equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,NaN,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,NaN,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,NaN,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,True,NaN
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,True,NaN
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,True,NaN
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,NaN,True,NaN


In [153]:
df_property = df_property.rename(columns={
    "referencia":             "property_native_id",
    "superficie construida":  "superficie_construida",
    "superficie útil":        "superficie_util",
    "baños":                  "banos",
    "conservación":           "estado_conservacion",
    "cocina equipada":        "cocina_equipada"
})

In [158]:
df_property.isna().sum()

barrio                     31
distrito                   31
latitude                    0
longitude                   0
property_native_id          0
superficie_construida      17
superficie_util            15
habitaciones                9
banos                       0
planta                      0
estado_conservacion         0
antigüedad               2505
amueblado                   0
cocina_equipada          3002
dtype: int64

In [159]:
df_property = df_property.replace({np.nan: None})

In [161]:
df_property

,barrio,distrito,latitude,longitude,property_native_id,superficie_construida,superficie_util,habitaciones,banos,planta,estado_conservacion,antigüedad,amueblado,cocina_equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,None,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,None,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,None,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,True,None
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,True,None
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,True,None
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,None,True,None


Contruimos la lista de tuplas para insertar

In [163]:
property_records = []
for _, row in df_property.iterrows():
    loc_id = loc_map.get((row.barrio, row.distrito))
    age_id = age_map.get(row.antigüedad)

    habitaciones = int(row.habitaciones) if row.habitaciones is not None else None
    banos        = int(row.banos)         if row.banos        is not None else None
    planta       = int(row.planta)        if row.planta       is not None else None

    property_records.append((
        loc_id,
        row.latitude,
        row.longitude,
        row.property_native_id,
        row.superficie_construida,
        row.superficie_util,
        habitaciones,
        banos,
        planta,
        row.estado_conservacion,
        age_id,
        row.amueblado,
        row.cocina_equipada
    ))

In [164]:
property_records

[(1,
  40.4342811,
  -3.6862864,
  '6013',
  272.0,
  272.0,
  4,
  5,
  6,
  'A estrenar',
  None,
  'No',
  'True'),
 (2,
  40.4386119,
  -3.6999008,
  '3893',
  74.0,
  74.0,
  1,
  1,
  6,
  'En buen estado',
  None,
  'Si',
  'True'),
 (1,
  40.4352372,
  -3.6835148,
  '4411',
  101.0,
  100.0,
  3,
  1,
  2,
  'En buen estado',
  None,
  'Si',
  'False'),
 (3,
  40.4290471,
  -3.6664739,
  '20815',
  65.0,
  50.0,
  3,
  1,
  6,
  'En buen estado',
  1,
  'Si',
  'Independiente'),
 (4,
  40.4226065,
  -3.6991962,
  '9635001141/9635',
  83.0,
  77.0,
  2,
  2,
  5,
  'En buen estado',
  1,
  'Si',
  'False'),
 (5,
  40.4446662,
  -3.7879384,
  'Pza_2dvac',
  90.0,
  90.0,
  2,
  1,
  2,
  'En buen estado',
  None,
  'No',
  'False'),
 (6,
  40.4430413,
  -3.5754374,
  '2209/48',
  81.0,
  62.0,
  1,
  1,
  0,
  'En buen estado',
  None,
  'No',
  'False'),
 (4,
  40.425045,
  -3.6990557,
  'EAT: 3062',
  28.0,
  28.0,
  1,
  1,
  0,
  'A estrenar',
  1,
  'Si',
  'Equipada'),
 (7,

In [167]:
sql_property = """
        INSERT INTO property
        (location_id, latitude, longitude,
        property_native_id,
        superficie_construida, superficie_util,
        habitaciones, banos, planta,
        estado_conservacion, age_range_id,
        amueblado, cocina_equipada)
        VALUES
        (%s,%s,%s,
        %s,
        %s,%s,
        %s,%s,%s,
        %s,%s,
        %s,%s)
    """

In [168]:
cursor.executemany(sql_property, property_records)

DataError: 1406 (22001): Data too long for column 'property_native_id' at row 59

### Tabla ``listing``

### Tabla `rental_term`

### Tabla `energy_certificate`

### Tabla `property_feature`

In [47]:
columnas2 = df_sale.columns.tolist()
columnas2

['price_eur',
 'barrio',
 'distrito',
 'latitude',
 'longitude',
 'energy_consumption_rating',
 'energy_consumption_value',
 'energy_emissions_rating',
 'energy_emissions_value',
 'adaptado a personas con movilidad reducida',
 'aire acondicionado',
 'amueblado',
 'antigüedad',
 'ascensor',
 'balcón',
 'baños',
 'calefacción',
 'chimenea',
 'conservación',
 'exterior',
 'garaje',
 'habitaciones',
 'jardín',
 'piscina',
 'planta',
 'portero automático',
 'puerta blindada',
 'sistema de seguridad',
 'superficie construida',
 'superficie útil',
 'terraza',
 'trastero',
 'vidrios dobles']

In [15]:
# Convertir DataFrame a lista de tuplas
datos = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas)
    for _, row in df.iterrows()
]

In [28]:
# Convertir DataFrame a lista de tuplas
datos2 = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas2)
    for _, row in df2.iterrows()
]

In [18]:
# Insertar todos los datos de golpe
cursor.executemany(sql, datos)
conn.commit()


In [29]:
# Insertar todos los datos de golpe
cursor2.executemany(sql2, datos2)
conn2.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'a personas con movilidad reducida, Agua, Aire acondicionado, Amueblado, Antigüe' at line 1

In [19]:
cursor.execute("SHOW TABLES")
for (table_name,) in cursor.fetchall():
    print(table_name)

countries


Insertar la tabla location

In [48]:
df_sale[["barrio","distrito","latitude","latitude"]]

,barrio,distrito,latitude,latitude
0,Recoletos,Salamanca,40.422116,40.422116
1,Universidad-Malasaña,Centro,40.422312,40.422312
2,Goya,Salamanca,40.424364,40.424364
3,Opañel,Carabanchel,40.389100,40.389100
4,Rejas,San Blas,40.442042,40.442042
...,...,...,...,...
2997,Portazgo,Puente de Vallecas,40.383712,40.383712
2998,Pueblo Nuevo,Ciudad Lineal,40.426333,40.426333
2999,Valdemarín,Moncloa-Aravaca,40.465343,40.465343
3000,Trafalgar,Chamberí,40.435597,40.435597


In [49]:
df_location = df_sale[["barrio","distrito","latitude","longitude"]]
df_location

,barrio,distrito,latitude,longitude
0,Recoletos,Salamanca,40.422116,-3.683149
1,Universidad-Malasaña,Centro,40.422312,-3.706137
2,Goya,Salamanca,40.424364,-3.670577
3,Opañel,Carabanchel,40.389100,-3.719900
4,Rejas,San Blas,40.442042,-3.575920
...,...,...,...,...
2997,Portazgo,Puente de Vallecas,40.383712,-3.650452
2998,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205
2999,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967
3000,Trafalgar,Chamberí,40.435597,-3.700479


In [50]:
columnas = df_location.columns.to_list()
columnas

['barrio', 'distrito', 'latitude', 'longitude']

In [51]:
# Convertir DataFrame a lista de tuplas
location = [
    tuple(row[col] for col in columnas)
    for _, row in df_location.iterrows()
]

In [52]:
location

[('Recoletos', 'Salamanca', 40.4221164, -3.6831486),
 ('Universidad-Malasaña', 'Centro', 40.422312, -3.706137),
 ('Goya', 'Salamanca', 40.4243638, -3.6705767),
 ('Opañel', 'Carabanchel', 40.3891, -3.7199),
 ('Rejas', 'San Blas', 40.4420421, -3.5759197),
 ('Lista', 'Salamanca', 40.4295857, -3.6735977),
 ('Guindalera', 'Salamanca', 40.4390839, -3.6613366),
 ('Embajadores-Lavapiés', 'Centro', 40.4129747, -3.7059398),
 ('Lucero', 'Latina', 40.4025977, -3.7530822),
 ('Goya', 'Salamanca', 40.4241, -3.67855),
 ('Recoletos', 'Salamanca', 40.4226281, -3.6831171),
 ('El Plantío', 'Moncloa-Aravaca', 40.4767, -3.82927),
 ('Castellana', 'Salamanca', 40.4277347, -3.685881),
 ('Las Águilas', 'Latina', 40.377671, -3.7731945),
 ('Lista', 'Salamanca', 40.4318763, -3.6736524),
 ('Palacio', 'Centro', 40.4118684, -3.7176348),
 ('El Cañaveral', 'Vicálvaro', 40.411542, -3.5447233),
 ('Lista', 'Salamanca', 40.4334893, -3.6773472),
 ('Guindalera', 'Salamanca', 40.434233, -3.6672526),
 ('Guindalera', 'Salamanca

Es importante que los datos estén en el formato adecuado para poder cargar la base de datos, tampoco podemos tener valores nan

In [56]:
sql_location = """
    INSERT INTO location (barrio, distrito, latitude, longitude)
    VALUES (%s, %s, %s, %s)
"""
cursor2.executemany(sql_location, location)
conn2.commit()

Se seguirá la misma estrategia de trabajo expuesta para cargar la base de datos cuando los datos extraidos esten limpios, teniendo en cuenta además que son dos fichero, uno para ventas y otro para alquileres

insertados datos de la tabla location del df de sale

### Tabla property de `sale`

In [73]:
loc_db = pd.read_sql(
    "SELECT location_id, latitude, longitude FROM location", conn2)

C:\Users\marta\AppData\Local\Temp\ipykernel_25340\400070013.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  loc_db = pd.read_sql(


In [77]:
loc_db.short(by="location_id")

AttributeError: 'DataFrame' object has no attribute 'short'

In [69]:
# recuperar ids
query_ids_location = """
SELECT location_id, latitude, longitude
FROM   location
WHERE  (latitude, longitude) IN (%s)
"""

In [71]:
cursor2.executemany(query_ids_location)

TypeError: CMySQLCursor.executemany() missing 1 required positional argument: 'seq_params'

In [60]:
cols_loc = ["barrio","distrito","latitude","longitude"]
df_loc   = df_sale[cols_loc].copy()

In [61]:
start_id  = cursor2.lastrowid - len(df_loc) + 1
start_id

-3000

In [65]:
cursor2.lastrowid

1

In [62]:
import numpy as np

In [63]:
df_loc["location_id"] = np.arange(start_id, start_id + len(df_loc))


In [64]:
df_loc["location_id"]

0      -3000
1      -2999
2      -2998
3      -2997
4      -2996
        ... 
2997      -3
2998      -2
2999      -1
3000       0
3001       1
Name: location_id, Length: 3002, dtype: int32